光流（Optical Flow）是指在图像序列中，由于物体运动导致的像素亮度模式的表观移动。它描述了图像中像素点的速度和方向信息，是计算机视觉中用于运动估计、物体跟踪和三维重建等领域的重要工具。

**数学逻辑与计算方法：**

1. **亮度恒常性假设（Brightness Constancy Assumption）**

   假设物体的亮度在运动过程中保持不变，即在时间 $t$ 和 $t+\Delta t$ 时刻，同一物体点的亮度值相等：

   $$
   I(x, y, t) = I(x + \Delta x, y + \Delta y, t + \Delta t)
   $$

2. **泰勒级数展开**

   对上述等式右边进行泰勒展开，忽略高阶项：

   $$
   I(x + \Delta x, y + \Delta y, t + \Delta t) \approx I(x, y, t) + \frac{\partial I}{\partial x} \Delta x + \frac{\partial I}{\partial y} \Delta y + \frac{\partial I}{\partial t} \Delta t
   $$

3. **建立光流方程**

   将展开式代入亮度恒常性假设，并消去 $I(x, y, t)$：

   $$
   \frac{\partial I}{\partial x} \Delta x + \frac{\partial I}{\partial y} \Delta y + \frac{\partial I}{\partial t} \Delta t = 0
   $$

   定义光流矢量 $(u, v)$：

   $$
   u = \frac{\Delta x}{\Delta t}, \quad v = \frac{\Delta y}{\Delta t}
   $$

   得到**光流基本约束方程**：

   $$
   \frac{\partial I}{\partial x} u + \frac{\partial I}{\partial y} v + \frac{\partial I}{\partial t} = 0
   $$

4. **不适定性问题（Aperture Problem）**

   由于一个方程有两个未知数 $u$ 和 $v$，方程不可直接求解。这就是所谓的孔径问题，需要额外的约束条件。

5. **求解方法**

   - **Lucas-Kanade 方法**

     假设在小的邻域内（如 $3 \times 3$ 窗口），光流是恒定的。利用窗口内所有像素的梯度信息，构建一个超定方程组，通过最小二乘法求解。

     矩阵形式：

     $$
     A \begin{bmatrix} u \\ v \end{bmatrix} = -b
     $$

     其中，

     $$
     A = \begin{bmatrix}
     I_{x1} & I_{y1} \\
     I_{x2} & I_{y2} \\
     \vdots & \vdots \\
     I_{xn} & I_{yn}
     \end{bmatrix}, \quad
     b = \begin{bmatrix}
     I_{t1} \\
     I_{t2} \\
     \vdots \\
     I_{tn}
     \end{bmatrix}
     $$

     通过求解：

     $$
     \begin{bmatrix} u \\ v \end{bmatrix} = -(A^TA)^{-1}A^Tb
     $$

   - **Horn-Schunck 方法**

     引入全局光滑性约束，假设整个图像的光流场是平滑的。通过最小化能量函数，得到一组偏微分方程，迭代求解。

     能量函数：

     $$
     E = \iint \left[ \left( \frac{\partial I}{\partial x} u + \frac{\partial I}{\partial y} v + \frac{\partial I}{\partial t} \right)^2 + \alpha^2 \left( \left| \nabla u \right|^2 + \left| \nabla v \right|^2 \right) \right] dx dy
     $$

     其中，$\alpha$ 是平滑参数。

6. **计算步骤**

   - **计算梯度**：计算图像的空间梯度 $\frac{\partial I}{\partial x}$、$\frac{\partial I}{\partial y}$ 和时间梯度 $\frac{\partial I}{\partial t}$。
   - **构建方程**：根据选定的方法（如 Lucas-Kanade 或 Horn-Schunck）构建方程组。
   - **求解光流**：使用最小二乘法或迭代算法求解光流矢量 $(u, v)$。
   - **结果优化**：对计算得到的光流场进行平滑或滤波，提升结果的可靠性。

**总结**

光流法的核心是基于亮度恒常性假设，利用图像的空间和时间梯度信息，建立光流约束方程。由于直接求解存在不适定性，需要引入额外的约束条件，如局部光流恒定（Lucas-Kanade）或全局光滑性（Horn-Schunck）。通过这些方法，可以有效地计算出图像序列中像素的运动信息，为后续的视觉任务提供基础。

In [1]:
# 导入必要的库
import cv2
import numpy as np

   - `cv2` 是 OpenCV 的 Python 接口，用于计算机视觉任务。
   - `numpy` 用于数值计算和数组操作。

3. **打开视频文件：**

In [2]:
cap = cv2.VideoCapture("/opt/videos/highway_side_01.mp4")

   - `VideoCapture` 对象用于读取视频文件或摄像头输入。

4. **读取第一帧并进行预处理：**


In [3]:
# 读取第一帧并将其转换为灰度图
ret, prev_frame = cap.read()
if not ret:
    print("Error reading the first frame.")
    cap.release()
    exit(1)

# 缩放比例
ratio = 0.15

# 缩小第一帧的尺寸
prev_frame = cv2.resize(prev_frame, (0, 0), fx=ratio, fy=ratio)
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

   - `ret, frame1 = cap.read()` 读取视频的第一帧。
   - `prvs` 保存转换为灰度图的第一帧，用于后续光流计算。
   - `hsv` 初始化一个与帧大小相同的零矩阵，用于存储 HSV 图像，其中饱和度（S 通道）设置为最大值 255。

5. **循环处理每一帧：**


In [4]:
while cap.isOpened():
    # 读取下一帧
    ret, frame = cap.read()
    if not ret:
        break

    # 缩小帧的尺寸
    frame = cv2.resize(frame, (0, 0), fx=ratio, fy=ratio)

    # 转换为灰度图
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 计算光流，获取像素的变化情况
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 
                                        0.5,    # 图像金字塔缩放因子
                                        3,      # 金字塔层数
                                        15,     # 窗口尺寸
                                        3,      # 迭代次数
                                        5,      # 多项式标准差
                                        1.2,    # 用于额外图像的高斯标准差
                                        0)      # 标志位，设置为 0 表示默认

    # 计算光流的大小和角度, 并转换为极坐标表示
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # 创建HSV图像，设置运动方向和强度
    hsv = np.zeros_like(frame)
    hsv[..., 1] = 255  # 饱和度设为最大值
    hsv[..., 0] = ang * 180 / np.pi / 2  # 方向角度转换为 [0, 180] 范围
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)  # 根据运动强度设置亮度

    # 将HSV转换为BGR格式，便于显示
    bgr_display = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    # 显示结果
    bgr_display = cv2.resize(bgr_display, (800, 600))
    cv2.imshow('Movement Detection in HSV', bgr_display)

    # 按下 'q' 键退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # 更新前一帧
    prev_gray = gray
    
cap.release()
cv2.destroyAllWindows()

<center>
<img src="res/hsv01.png" alt="HSV description"/>
</center>

<center>
<img src="res/hsv02.png" alt="HSV description"/>
</center>

因为RGB的颜色范围是
* R [0-255]
* G [0-255]
* B [0-255]

而HSV的颜色空间则是
* H [0-360]
* S [0-255]
* V [0-255]

**总结：**

这段代码主要用于计算视频中连续帧之间的光流，并以颜色编码的方式可视化光流，显示物体在视频中的运动方向和速度。颜色的色调表示运动方向，亮度表示运动速度。

**应用场景：**

- **运动检测**：识别视频中移动的物体。
- **对象跟踪**：跟踪特定物体在视频中的运动轨迹。
- **视频稳定**：通过估计相机运动来稳定视频。
- **交通分析**：分析交通流量和车辆速度。